<a href="https://colab.research.google.com/github/Davjot/PlantDoc/blob/main/Image_plantdoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

In [24]:
!wget --no-check-certificate \
   https://storage.googleapis.com/tensorflow/keras-applications/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-04-04 19:03:10--  https://storage.googleapis.com/tensorflow/keras-applications/inception_resnet_v2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219055592 (209M) [application/octet-stream]
Saving to: ‘inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5’

inception_resnet_v2 100%[===================>] 208.91M  32.6MB/s    in 6.7s    

2023-04-04 19:03:17 (31.0 MB/s) - ‘inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [219055592/219055592]



In [25]:
from tensorflow.keras.applications import InceptionResNetV2

local_weights_file = '/content/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionResNetV2(
    input_shape=(150, 150, 3), include_top=False, weights='imagenet')
pre_trained_model.load_weights(local_weights_file)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = '/content/drive/MyDrive/Plantdoc'
train_dir = os.path.join(base_dir)
validation_dir = os.path.join(base_dir)

In [4]:
train_apple_rust_leaf_dir = os.path.join(train_dir, 'Apple rust leaf')
validation_apple_rust_leaf_dir = os.path.join(validation_dir, 'Apple rust leaf')
train_blueberry_leaf_dir = os.path.join(train_dir, 'Blueberry leaf')
validation_blueberry_leaf_dir = os.path.join(validation_dir, 'Blueberry leaf')
#train_apple_rust_leaf_fnames = os.listdir(train_apple_rust_leaf_dir) 
#training set
train_apple_leaf_dir = os.path.join(train_dir, 'Apple leaf')
train_apple_scab_leaf_dir = os.path.join(train_dir, 'Apple scab leaf')
train_bell_pepper_leaf_dir = os.path.join(train_dir, 'Bell pepper leaf')
train_bell_pepper_leaf_spot_dir = os.path.join(train_dir, 'Bell pepper leaf spot')
train_cherry_leaf_dir = os.path.join(train_dir, 'Cherry leaf')
train_corn_gray_leaf_spot_dir = os.path.join(train_dir, 'Corn Gray leaf spot')

#validation set
validation_apple_leaf_dir = os.path.join(validation_dir, 'Apple leaf')
validation_apple_scab_leaf_dir = os.path.join(validation_dir, 'Apple scab leaf')
validation_bell_pepper_leaf_dir = os.path.join(validation_dir, 'Bell pepper leaf')
validation_bell_pepper_leaf_spot_dir = os.path.join(validation_dir, 'Bell pepper leaf spot')
validation_cherry_leaf_dir = os.path.join(validation_dir, 'Cherry leaf')
validation_corn_gray_leaf_spot_dir = os.path.join(validation_dir, 'Corn Gray leaf spot')

In [5]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=1,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode='categorical')

Found 80 images belonging to 8 classes.
Found 80 images belonging to 8 classes.


In [6]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [19]:
# create the base pre-trained model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = (150, 150, 3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(8, activation='softmax')(x)

In [20]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [21]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['acc'])

In [26]:
import tensorflow as tf
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5', save_weights_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/logdir_plantdoc'),
]

In [27]:
history = model.fit(
      train_generator,
      epochs=5,
      validation_data=validation_generator,
      verbose=2,
      callbacks = [my_callbacks])

Epoch 1/5
80/80 - 32s - loss: 1.2762 - acc: 0.5625 - val_loss: 1.3131 - val_acc: 0.5500 - 32s/epoch - 396ms/step
Epoch 2/5
80/80 - 23s - loss: 1.2790 - acc: 0.5250 - val_loss: 1.3677 - val_acc: 0.5125 - 23s/epoch - 290ms/step
Epoch 3/5
80/80 - 33s - loss: 1.2379 - acc: 0.5875 - val_loss: 1.1165 - val_acc: 0.6000 - 33s/epoch - 408ms/step
Epoch 4/5
80/80 - 32s - loss: 1.1262 - acc: 0.6000 - val_loss: 0.8695 - val_acc: 0.7125 - 32s/epoch - 406ms/step
Epoch 5/5
80/80 - 32s - loss: 1.0398 - acc: 0.6125 - val_loss: 1.0255 - val_acc: 0.6625 - 32s/epoch - 402ms/step
